In [1]:
import os
import json
import random

In [3]:
with open(os.path.join('data', 'multi_frame', 'test_eval.json')) as f:
    test_data = json.load(f)

# behavior_test_data = [item for item in test_data if item[0]['Q'] == 'Predict the behavior of the ego vehicle.']

# len(behavior_test_data)

In [2]:
with open(os.path.join('data', 'QA_dataset_nus', 'v1_1_train_nus_final.json')) as f:
    data = json.load(f)
with open(os.path.join('data', 'QA_dataset_nus', 'v1_1_val_nus_q_only.json')) as f:
    test_data = json.load(f)

In [3]:
scene_tokens = list(data.keys())
random.shuffle(scene_tokens)
train_data = scene_tokens[:int(len(data)*0.95)]
val_data = scene_tokens[int(len(data)*0.95):]
train_data = {scene_token: data[scene_token] for scene_token in train_data}
val_data = {scene_token: data[scene_token] for scene_token in val_data}
len(train_data), len(val_data), len(test_data)

(661, 35, 149)

In [23]:

# with open(os.path.join('data','multi_frame', 'test_eval.json')) as f:
#   data = json.load(f)

scene_tokens = list(data.keys())
random.shuffle(scene_tokens)
train_data = scene_tokens[:int(len(data)*0.85)]
val_data = scene_tokens[int(len(data)*0.85):int(len(data)*0.9)]
test_data = scene_tokens[int(len(data)*0.9):int(len(data)*0.95)]
rag_data = scene_tokens[int(len(data)*0.95):]

train_data = {scene_token: data[scene_token] for scene_token in train_data}
val_data = {scene_token: data[scene_token] for scene_token in val_data}
test_data = {scene_token: data[scene_token] for scene_token in test_data}
rag_data = {scene_token: data[scene_token] for scene_token in rag_data}

len(train_data), len(val_data), len(test_data), len(rag_data)

(591, 35, 35, 35)

# Number of Frames in DriveLM

In [4]:
num_frames_train = sum([len(train_data[scene_token]['key_frames']) for scene_token in train_data])
num_frames_val = sum([len(val_data[scene_token]['key_frames']) for scene_token in val_data])
num_frames_test = sum([len(test_data[scene_token]['key_frames']) for scene_token in test_data])
total_frames = num_frames_train + num_frames_val + num_frames_test 
num_frames_train, num_frames_val, num_frames_test, (num_frames_train / total_frames), (num_frames_val / total_frames), (num_frames_test / total_frames)

(3853, 219, 799, 0.7910080065694929, 0.04495996715253541, 0.16403202627797167)

# Calculating # of QA's in Dataset

In [5]:
from collections import defaultdict

def count_qa(d):

    num_qa = 0
    qa = set()
    question_types = defaultdict(int)
    
    for scene_token in d:
    
      for frame_token in d[scene_token]['key_frames']:
    
        qa = qa.union(set(d[scene_token]['key_frames'][frame_token]['QA'].keys()))
    
        for q in d[scene_token]['key_frames'][frame_token]['QA']:
    
          question_types[q] += len(d[scene_token]['key_frames'][frame_token]['QA'][q])
          num_qa += len(d[scene_token]['key_frames'][frame_token]['QA'][q])

    return num_qa, qa, question_types

train_qa = count_qa(train_data)
val_qa = count_qa(val_data)
test_qa = count_qa(test_data)
total_qa = (train_qa[0] + val_qa[0] + test_qa[0])

train_qa, val_qa, test_qa, train_qa[0] / total_qa, val_qa[0] / total_qa, test_qa[0] / total_qa

((555576,
  {'behavior', 'perception', 'planning', 'prediction'},
  defaultdict(int,
              {'perception': 306580,
               'prediction': 233588,
               'planning': 11555,
               'behavior': 3853})),
 (32540,
  {'behavior', 'perception', 'planning', 'prediction'},
  defaultdict(int,
              {'perception': 18380,
               'prediction': 13284,
               'planning': 657,
               'behavior': 219})),
 (15480,
  {'behavior', 'perception', 'planning', 'prediction'},
  defaultdict(int,
              {'perception': 1543,
               'prediction': 6539,
               'planning': 6599,
               'behavior': 799})),
 0.9204434754372129,
 0.053910231346794873,
 0.025646293215992155)

In [6]:
def save_mf_pairs(data):

    multi_frame_qi_pairs = []
    num_questions  = 0

    for scene_token in list(data.keys()):
    
      scene_frames = data[scene_token]['key_frames']
    
      for frame_token in scene_frames:
    
        frame = scene_frames[frame_token]
        frame_data_qa = frame['QA']
        frame_data_qa = frame_data_qa['perception'] + frame_data_qa['prediction'] + frame_data_qa['planning'] + frame_data_qa['behavior']
    
        for i, question in enumerate(frame_data_qa):
    
          image_paths = scene_frames[frame_token]['image_paths']        
          question['id'] = f'{scene_token}_{frame_token}_{i}'
          multi_frame_qi_pairs.append((question, image_paths))

    for i in range(len(multi_frame_qi_pairs)):

      new_cam_set = {cam: '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_nus_imgs_train' + multi_frame_qi_pairs[i][1][cam][2:] for cam in multi_frame_qi_pairs[i][1]}
      multi_frame_qi_pairs[i] = (multi_frame_qi_pairs[i][0], dict(new_cam_set))

    return multi_frame_qi_pairs

def save_mf_behavior_pairs(data):

    multi_frame_qi_pairs = []
    num_questions  = 0

    for scene_token in list(data.keys()):
    
      scene_frames = data[scene_token]['key_frames']
    
      for frame_token in scene_frames:
    
        frame = scene_frames[frame_token]
    
        for q in frame['QA']:
    
          qa_list = frame['QA'][q]
          image_paths = scene_frames[frame_token]['image_paths']

          if q == 'behavior':
    
              for question in qa_list:
        
                multi_frame_qi_pairs.append((question, image_paths))

    for i in range(len(multi_frame_qi_pairs)):

      new_cam_set = {cam: '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_nus_imgs_train' + multi_frame_qi_pairs[i][1][cam][2:] for cam in multi_frame_qi_pairs[i][1]}
      multi_frame_qi_pairs[i] = (multi_frame_qi_pairs[i][0], dict(new_cam_set))

    return multi_frame_qi_pairs

    



multi_frame_train = save_mf_pairs(train_data)
multi_frame_val = save_mf_pairs(val_data)
multi_frame_test = save_mf_pairs(test_data)
random.shuffle(multi_frame_train), random.shuffle(multi_frame_val)
len(multi_frame_train), len(multi_frame_val), len(multi_frame_test), multi_frame_test[:10]
# multi_frame_test_behavior[:10]

(555576,
 32540,
 15480,
 [({'Q': 'What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.',
    'A': '',
    'C': None,
    'con_up': None,
    'con_down': None,
    'cluster': None,
    'layer': None,
    'tag': [2],
    'id': 'b789de07180846cc972118ee6d1fb027_b0e6fd5561454b2789c853e5350557a8_0'},
   {'CAM_FRONT': '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_nus_imgs_train/nuscenes/samples/CAM_FRONT/n008-2018-09-18-14-35-12-0400__CAM_FRONT__1537295990612404.jpg',
    'CAM_FRONT_LEFT': '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_nus_imgs_train/nuscenes/samples/CAM_FRONT_LEFT/n008-2018-09-18-14-35-12-0400__CAM_FRONT_LEFT__1537295990604799.jpg',
    'CAM_FRONT_RIGHT': '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_nus_imgs_train/nuscenes/samples/CAM_FRONT_RIGHT/n008-2018-09-18-14-35-12-0400__CAM_FRONT_RIGHT__1537295990620482.jpg',
    'CAM_BACK': '/home/cvrr/Desktop/EfficientDriveLM/data/drivelm_n

In [7]:
multi_frame_dset = [(multi_frame_train, 'multi_frame_train_DriveLM-challenge.json'),
 (multi_frame_val, 'multi_frame_val_DriveLM-challenge.json')]
# multi_frame_dset = [(multi_frame_test, 'multi_frame_test_challenge_only_q.json')]

for dset, fname in multi_frame_dset:

  with open(os.path.join('../EM-VLM4AD', 'data', 'multi_frame', fname), 'w') as f:
    json.dump(dset, f, indent=6)


# Pair Each Question with its Corresponding Image

In [ ]:
CAM_FRONT, CAM_FRONT_LEFT, CAM_FRONT_RIGHT = 'CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT'
CAM_BACK, CAM_BACK_LEFT, CAM_BACK_RIGHT = 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT'
front, front_left, front_right = 'front', 'front left', 'front right'
back, back_left, back_right = 'back', 'back left', 'back right'

cam_list = [(CAM_FRONT_LEFT, front_left), (CAM_FRONT_RIGHT, front_right), (CAM_FRONT, front),
            (CAM_BACK_LEFT, back_left), (CAM_BACK_RIGHT, back_right), (CAM_BACK, back)]

single_frame_qi_pairs = []
multi_frame_qi_pairs = []
num_questions  = 0


for scene_token in list(data.keys()):

  scene_frames = data[scene_token]['key_frames']

  for frame_token in scene_frames:

    frame = scene_frames[frame_token]

    for q in frame['QA']:

      qa_list = frame['QA'][q]
      image_paths = scene_frames[frame_token]['image_paths']


      for question in qa_list:

        num_cams = set()

        for cam, direc in cam_list:
          if cam in question['Q'] or direc in question['Q'].lower():
            num_cams.add((cam, direc))
          if cam in question['A'] or direc in question['A'].lower():
            num_cams.add((cam, direc))

        num_cams = list(num_cams)

        if len(num_cams) == 1:
          # print(question['Q'], question['A'])
          single_frame_qi_pairs.append((question, image_paths[num_cams[0][0]]))

        # if CAM_FRONT in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT]))
        # elif CAM_FRONT_RIGHT in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT_RIGHT]))
        # elif CAM_FRONT_LEFT in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT_LEFT]))
        # elif CAM_BACK in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK]))
        # elif CAM_BACK_RIGHT in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK_RIGHT]))
        # elif CAM_BACK_LEFT in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK_LEFT]))
        # elif front_right in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT_RIGHT]))
        # elif front_left in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT_LEFT]))
        # elif front in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_FRONT]))
        # elif back_right in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK]))
        # elif back_left in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK_RIGHT]))
        # elif back in question['Q']:
        #   single_frame_qi_pairs.append((question, image_paths[CAM_BACK_LEFT]))

        multi_frame_qi_pairs.append((question, image_paths))

        num_questions += 1

num_questions, len(single_frame_qi_pairs), len(single_frame_qi_pairs)/num_questions

(377983, 198595, 0.5254072273091647)

In [ ]:
# Update the paths for the image pairs
for i in range(len(single_frame_qi_pairs)):

  single_frame_qi_pairs[i] = (single_frame_qi_pairs[i][0], os.path.join('drive', 'MyDrive', 'DriveLM', 'data') + single_frame_qi_pairs[i][1][2:])

for i in range(len(multi_frame_qi_pairs)):

  new_cam_set = {cam: os.path.join('drive', 'MyDrive', 'DriveLM', 'data') + multi_frame_qi_pairs[i][1][cam][2:] for cam in multi_frame_qi_pairs[i][1]}
  multi_frame_qi_pairs[i] = (multi_frame_qi_pairs[i][0], dict(new_cam_set))


len(single_frame_qi_pairs), len(multi_frame_qi_pairs)

(198595, 377983)

# Save QA-Image Pairs

In [ ]:
import random

random.shuffle(single_frame_qi_pairs), random.shuffle(multi_frame_qi_pairs)

single_frame_train = single_frame_qi_pairs[:int(len(single_frame_qi_pairs)*0.9)]
single_frame_val = single_frame_qi_pairs[int(len(single_frame_qi_pairs)*0.9):int(len(single_frame_qi_pairs)*0.95)]
single_frame_test = single_frame_qi_pairs[int(len(single_frame_qi_pairs)*0.95):]


multi_frame_train = multi_frame_qi_pairs[:int(len(multi_frame_qi_pairs)*0.9)]
multi_frame_val = multi_frame_qi_pairs[int(len(multi_frame_qi_pairs)*0.9):int(len(multi_frame_qi_pairs)*0.95)]
multi_frame_test = multi_frame_qi_pairs[int(len(multi_frame_qi_pairs)*0.95):]

# os.mkdir(os.path.join('drive', 'MyDrive', 'DriveLM', 'single_frame'))
# os.mkdir(os.path.join('drive', 'MyDrive', 'DriveLM', 'multi_frame'))

single_frame_dset = [(single_frame_train, 'single_frame_train.json'),
 (single_frame_val, 'single_frame_val.json'), (single_frame_test, 'single_frame_test.json')]
multi_frame_dset = [(multi_frame_train, 'multi_frame_train.json'),
 (multi_frame_val, 'multi_frame_val.json'), (multi_frame_test, 'multi_frame_test.json')]

for dset, fname in single_frame_dset:

  with open(os.path.join('drive', 'MyDrive', 'DriveLM', 'data', 'single_frame', fname), 'w') as f:
    json.dump(dset, f, indent=6)

for dset, fname in multi_frame_dset:

  with open(os.path.join('drive', 'MyDrive', 'DriveLM', 'data', 'multi_frame', fname), 'w') as f:
    json.dump(dset, f, indent=6)
